In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# keras module for building LSTM 
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import keras
import tensorflow as tf
from tensorflow.keras.utils import Sequence

if IN_COLAB:
  !pip install Keras-Preprocessing
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

import pandas as pd
import numpy as np
import string, os 

2023-04-12 09:48:59.336784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Text Generation with LSTM

In [3]:
# Get Data
train_text_file = keras.utils.get_file('train_text.txt', 'https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/reddit_wsb.csv')
train_text = pd.read_csv(train_text_file)
train_text.sample(10)

,title,score,id,url,comms_num,created,body,timestamp
8528,I cut my losses on 1/29 AMC options on Robinho...,1,l70kie,https://i.redd.it/5u1cwalki3e61.png,4,1.611878e+09,NaN,2021-01-29 02:00:06
35401,Xl fleet will soon take more than amc from me,35,lq83uq,https://i.redd.it/vto8ehjcd5j61.jpg,19,1.614080e+09,NaN,2021-02-23 13:33:09
864,Remember the kind of people behind Melvin Capital,3,l6wwwt,https://www.reddit.com/r/wallstreetbets/commen...,0,1.611870e+09,[Business Insider: Former SAC Capital Employee...,2021-01-28 23:44:03
19798,"AMC GME Yes, I am still holding💰 Will buy more...",1408,l9hl1m,https://i.redd.it/jz8vfbunhpe61.jpg,192,1.612144e+09,NaN,2021-02-01 03:52:17
43019,Holding,336,mc4sv4,https://i.redd.it/musbjsyc2zo61.jpg,35,1.616619e+09,NaN,2021-03-24 22:45:40
31048,What I'll be using to spend my GME when I sell...,88,ldlk7l,https://i.redd.it/mm20csax4rf61.jpg,14,1.612600e+09,NaN,2021-02-06 10:28:36
52842,77K PTRA YOLO (on margin),103,ozbwja,https://i.redd.it/7a1siaxp0sf71.png,27,1.628273e+09,NaN,2021-08-06 20:56:46
14576,The FACT that DeepFuckingValue might have star...,1,l732fa,https://www.reddit.com/r/wallstreetbets/commen...,0,1.611884e+09,Anywhere you look currently there are no polit...,2021-01-29 03:26:08
44092,Ride it or sell on next gap up?,9,mpur6o,https://i.redd.it/075l0wu28vs61.jpg,22,1.618316e+09,NaN,2021-04-13 15:07:09
10974,So uhhhh... time to Occupy Wall Street again? 🤔,1,l71i24,https://www.reddit.com/r/wallstreetbets/commen...,2,1.611880e+09,NaN,2021-01-29 02:32:50


In [4]:
TOKENS = 1000
OUTPUT_LENGTH = 25
OUT_DIM = 8
BATCH_SIZE = 32

sentence to text words

In [5]:
raw_text = train_text['body']
## Remove punctuation
raw_text = raw_text.dropna()
raw_text = raw_text.apply(lambda x: x.replace('[{}]'.format(string.punctuation), ''))
vocab = set()
sentences = []
for sentence in raw_text:
  current_sentence = text_to_word_sequence(sentence)
  sentences.append(current_sentence)
  vocab.update(current_sentence)
#vocab
#sentences
max_length = max([len(sentence) for sentence in sentences])

#### Tokenize

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
tokenizer

In [ ]:
tokenized_sentences = tokenizer.texts_to_sequences(sentences)
tokenized_sentences

#### Generate Sequences

In [7]:
## convert data to sequence of tokens 
def make_sequence(sentences):
    input_sequences = []
    for line in sentences:
        for i in range(1, len(line)):
            n_gram_sequence = line[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences

In [8]:
class sequenceGenerator(Sequence):
    def __init__(self, sequences, batch_size):
        self.index = 0
        self.sequences = sequences
        self.batch_size = batch_size

    def __len__(self):
        return len(self.files)
    
    #def on_epoch_end(self):
        # Update

    def __getitem__(self, index):
        for i in range(index, index + self.batch_size):
            sequence = self.sequences[i]
            x = sequence[:-1]
            y = sequence[-1]
            x = tokenizer.texts_to_sequences(x)
            x = pad_sequences(x, maxlen=max_length-1, padding='pre')
            y = ku.to_categorical(y, num_classes=TOKENS)
            yield x, y

In [9]:
sequences = make_sequence(sentences)
tok_seq = make_sequence(tokenized_sentences)
#sequences

In [1]:
sequences[1]

NameError: name 'sequences' is not defined

In [ ]:
tok_seq[1]

#### Dataset Preparation

In [26]:
# Tokenize and Pad
text_gen = sequenceGenerator(sequences, BATCH_SIZE)

### Create Model

### Embedding Layers

In [29]:
model = Sequential()
model.add(Embedding(input_dim=len(vocab),output_dim=OUT_DIM,input_length=max_length))
model.add(Flatten())
model.add(Dense(len(vocab), activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
print(model.summary())

: 

: 

In [ ]:
history = model.fit(text_gen, epochs=10, verbose=1, steps_per_epoch=len(text_gen)//BATCH_SIZE)

Generator


In [9]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_text['body'].dropna())


In [10]:
class CSVGenerator(Sequence):
    def __init__(self, file_path):
        self.file = file_path
        self.index = 0

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        file_path = os.path.join(self.folder_path, self.files[index])
        data = pd.read_csv(file_path)
        X = data.iloc[:, :-1].values
        y = data.iloc[:, -1].values
        print("X: ", X, "y: ", y)
        return X, y

In [11]:
tmp = CSVGenerator(train_text_file)

Text Vectorization with Keras

In [12]:
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=TOKENS,
    #output_mode='int',
    output_sequence_length=OUTPUT_LENGTH)


2023-04-12 09:27:05.557453: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
text_dataset = tf.data.Dataset.from_tensor_slices(train_text["body"].dropna())
max_features = 5000  # Maximum vocab size.
max_len = 4  # Sequence length to pad the outputs to.

vectorize_layer.adapt(text_dataset.batch(64))

vect_text = vectorize_layer(text_dataset)
vect_text


ValueError: Exception encountered when calling layer 'text_vectorization' (type TextVectorization).

Attempt to convert a value (<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>) with an unsupported type (<class 'tensorflow.python.data.ops.from_tensor_slices_op.TensorSliceDataset'>) to a Tensor.

Call arguments received by layer 'text_vectorization' (type TextVectorization):
  • inputs=<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

### Clean Text

We can clean and prep our text here. The data cleanup we need is to:
<ul>
<li> Remove punctuation.
<li> Tokenize the text, as we did previously in NLP processing. 
<li> <b>Generate sequences of tokens.</b> This is the key to the LSTM model, we are structuring the data to be a sequence of tokens. Our model will attempt to predict the next token, which in this case is the next word in the sentence.
</ul>

In [ ]:
def get_sequence_of_tokens(corpus, tokenizer):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

strip_punct = train_text["body"].dropna().str.replace('[{}]'.format(string.punctuation), '')
inp_seq, total_words = get_sequence_of_tokens(strip_punct, Tokenizer())
inp_seq

/var/folders/px/vhm_920n7zx2wvqq_ht0q5tm0000gp/T/ipykernel_50997/955513030.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  strip_punct = train_text["body"].dropna().str.replace('[{}]'.format(string.punctuation), '')


KeyboardInterrupt: 

#### Dataset Prep - Padding and Targets

We also need to take the sequences and pad them, or make them all the same length. We will also create the targets - the next word in the sequence.

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_seq)

: 

: 

### Model

Now we model. The data that we made mirrors the construction of a sentence.
<ul>
<li> X features - the sentence up to this point. 
<li> Y target - the word(s) that should come next. 
</ul>

So, the model is effectively working to generate text just like a time series model works to predict the next value in a sequence of stock prices or hourly temperature. We train the model on, hopefully a large number of senteneces, where is sees many examples of "here are some words" (X values) and "here is the next word" (Y value). If we give it lots and lots of that training data, it should become better and better at determining what should come next, given the existing sentence. 

To do this well, we'd need a lot more data than we have, and much more time to train. We'd want to give the model enough data so that it can see lots and lots of examples of the same word in different contexts, and of similar contexts with different words. The patterns of language are really complex, so we need data that provides enough variation to demonstrate the patterns. 

The model is wrapped in a little function, so we can make a model to output a different number of words with more convenience.

#### Embedding Layer

We also use an embedding layer here, which accepts our enocoded inputs. 

In [ ]:
def create_model(max_sequence_len, output_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layers - LSTM Layer
    model.add(LSTM(100, return_sequences = True))
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, OUTPUT_LENGTH)
model.summary()

In [ ]:
# Train Model
model.fit(predictors, label, epochs=100, verbose=5)

### Predictions

We can create a little function to generate text. We can give it a seed text, and it will generate text based on that. We can also give it a number of words to generate, and it will generate that many words.

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:


print (generate_text("united states", 5, model, max_sequence_len))
print (generate_text("preident trump", 4, model, max_sequence_len))
print (generate_text("donald trump", 4, model, max_sequence_len))
print (generate_text("india and china", 4, model, max_sequence_len))
print (generate_text("new york", 4, model, max_sequence_len))
print (generate_text("science and technology", 5, model, max_sequence_len))

